<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# MAT281 - Laboratorio N°03





**Objetivo**: Aplicar técnicas avanzadas de manipulación y análisis de datos con pandas sobre un conjunto real de datos de contenido de Netflix, reforzando buenas prácticas y métodos eficientes sin recurrir a `groupby`, `merge`, `pivot`, ni `join`.



**Dataset**:

Trabajaremos con el archivo `netflix_titles.csv`, que contiene información sobre los títulos disponibles en la plataforma Netflix hasta el año 2021.

| Variable       | Clase     | Descripción                                                                 |
|----------------|-----------|------------------------------------------------------------------------------|
| show_id        | caracter  | Identificador único del título en el catálogo de Netflix.                   |
| type           | caracter  | Tipo de contenido: 'Movie' o 'TV Show'.                                     |
| title          | caracter  | Título del contenido.                                                       |
| director       | caracter  | Nombre del director (puede ser nulo).                                       |
| cast           | caracter  | Lista de actores principales (puede ser nulo).                              |
| country        | caracter  | País o países donde se produjo el contenido.                                |
| date_added     | fecha     | Fecha en la que el título fue agregado al catálogo de Netflix.              |
| release_year   | entero    | Año de lanzamiento original del título.                                     |
| rating         | caracter  | Clasificación por edad (por ejemplo: 'PG-13', 'TV-MA').                      |
| duration       | caracter  | Duración del contenido (minutos o número de temporadas para series).        |
| listed_in      | caracter  | Categorías o géneros en los que está clasificado el contenido.              |
| description    | caracter  | Breve sinopsis del contenido.                                               |




In [9]:
import pandas as pd

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...



### Parte 1: Limpieza y preparación

1. Revisar y describir el dataset:

   * ¿Cuántas filas y columnas tiene?
   * ¿Qué tipos de datos hay?
   * ¿Cuántos valores nulos hay por columna?

2. Transformar la columna `date_added` a tipo fecha.

3. Crear columnas auxiliares con `assign`:

   * Año (`year_added`)
   * Mes (`month_added`)



In [11]:
#FIXME


# 1. Filas y columnas
print(f"1) El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.\n")

# 2. Tipos de datos
print("2) Tipos de datos por columna:\n")
print(df.dtypes, "\n")

# 3. Valores nulos
print("3) Cantidad de valores nulos por columna:\n")
print(df.isna().sum(), "\n")

# 4. Transformar columna date_added
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# 5. Crear columnas auxiliares
df = df.assign(
    year_added = df['date_added'].dt.year,
    month_added = df['date_added'].dt.month
)

print("4) Ejemplo de columnas transformadas:\n")
print(df[['date_added','year_added','month_added']].head(10))


1) El dataset tiene 8807 filas y 14 columnas.

2) Tipos de datos por columna:

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
year_added             float64
month_added            float64
dtype: object 

3) Cantidad de valores nulos por columna:

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        98
release_year       0
rating             4
duration           3
listed_in          0
description        0
year_added        98
month_added       98
dtype: int64 

4) Ejemplo de columnas transformadas:

  date_added  year_added  month_added
0 2021-09-25      2021.0          9.0
1 20

## Parte 2: Técnicas avanzadas de pandas

4. Utilizar `.loc` para seleccionar películas (`type == 'Movie'`) que fueron agregadas después del año 2018.

5. Utilizar `str.contains()` y `str.extract()`:

   * Filtrar títulos que contienen la palabra 'love' (sin distinguir mayúsculas/minúsculas).
   * Extraer la duración en minutos para las películas desde la columna `duration`.

6. Aplicar `explode()` sobre la columna `listed_in` para obtener una fila por cada género.

7. Obtener un top 10 de géneros más frecuentes utilizando `value_counts()`.

8. Aplicar `where()` y `mask()` para marcar las películas de más de 120 minutos como contenido largo en una nueva columna.

9. Utilizar `.loc` para filtrar películas que cumplen con:

   * Más de 100 minutos de duración.
   * Rating igual a `'R'`.
   * País igual a `'United States'`.

10. Utilizar `.style` para formatear visualmente el top 10 de películas más largas.

In [13]:
#FIXME
import numpy as np

# 4) .loc: Movies agregadas después de 2018
movies_post2018 = df.loc[(df['type']=='Movie') & (df['date_added'].dt.year > 2018)]
print("4) Movies agregadas después de 2018:", movies_post2018.shape[0])
print(movies_post2018[['title','date_added']].head(5), "\n")

# 5) str.contains y str.extract
love_titles = df.loc[df['title'].str.contains('love', case=False, na=False), ['title','type']]
print("5.a) Títulos que contienen 'love':", love_titles.shape[0])
print(love_titles.head(5), "\n")

df['duration_min'] = pd.to_numeric(df['duration'].str.extract(r'(\d+)')[0], errors='coerce')
print("5.b) Ejemplo extracción de minutos:")
print(df[['duration','duration_min']].head(8), "\n")

# 6) explode en 'listed_in'
df_exploded = df.assign(listed_in=df['listed_in'].str.split(r',\s*', regex=True)).explode('listed_in')
df_exploded['listed_in'] = df_exploded['listed_in'].str.strip()
print("6) Explode 'listed_in':", df_exploded.shape)
print(df_exploded[['title','listed_in']].head(8), "\n")

# 7) Top 10 géneros por frecuencia
top10_genres = df_exploded['listed_in'].value_counts().head(10)
print("7) Top 10 géneros más frecuentes:\n", top10_genres, "\n")

# 8) where() y mask(): marcar movies >120 min como contenido largo
is_long = (df['type']=='Movie') & (df['duration_min'] > 120)
df['is_long_movie'] = False
df['is_long_movie'] = df['is_long_movie'].mask(is_long, True)         # mask
df['long_tag'] = pd.Series('short', index=df.index).where(~is_long, 'long')  # where
print("8) Conteo 'long' vs 'short':\n", df['long_tag'].value_counts(), "\n")

# 9) .loc: Movies >100 min, rating 'R', país incluye 'United States'
filt = (
    (df['type']=='Movie') &
    (df['duration_min'] > 100) &
    (df['rating'] == 'R') &
    (df['country'].str.contains('United States', na=False))
)
result_9 = df.loc[filt, ['title','duration_min','rating','country']]
print("9) Películas que cumplen las 3 condiciones:", result_9.shape[0])
print(result_9.head(10), "\n")

# 10) .style: Top 10 películas más largas
top10_longest = (df.loc[df['type']=='Movie', ['title','duration_min','rating','country']]
                   .sort_values('duration_min', ascending=False)
                   .head(10))
print("10) Top 10 películas más largas (tabla estilizada se ve en notebook):")
display(top10_longest.style.format({'duration_min':'{:.0f}'})
        .set_properties(**{'text-align':'left'})
        .set_table_styles([{'selector':'th','props':[('text-align','left')]}])
        .background_gradient(subset=['duration_min']))

4) Movies agregadas después de 2018: 3701
                               title date_added
0               Dick Johnson Is Dead 2021-09-25
6   My Little Pony: A New Generation 2021-09-24
7                            Sankofa 2021-09-24
9                       The Starling 2021-09-24
12                      Je Suis Karl 2021-09-23 

5.a) Títulos que contienen 'love': 196
                         title     type
25        Love on the Spectrum  TV Show
158    Love Don't Cost a Thing    Movie
159             Love in a Puff    Movie
206  LSD: Love, Sex Aur Dhokha    Movie
227                Really Love    Movie 

5.b) Ejemplo extracción de minutos:
    duration  duration_min
0     90 min          90.0
1  2 Seasons           2.0
2   1 Season           1.0
3   1 Season           1.0
4  2 Seasons           2.0
5   1 Season           1.0
6     91 min          91.0
7    125 min         125.0 

6) Explode 'listed_in': (19323, 17)
                   title               listed_in
0   Dick Johnson Is D

,title,duration_min,rating,country
4253,Black Mirror: Bandersnatch,312,TV-MA,United States
717,Headspace: Unwind Your Mind,273,TV-G,nan
2491,The School of Mischief,253,TV-14,Egypt
2487,No Longer kids,237,TV-14,Egypt
2484,Lock Your Girls In,233,TV-PG,nan
2488,Raya and Sakina,230,TV-14,nan
166,Once Upon a Time in America,229,R,"Italy, United States"
7932,Sangam,228,TV-14,India
1019,Lagaan,224,PG,"India, United Kingdom"
4573,Jodhaa Akbar,214,TV-14,India




### Pregunta Desafío

11. ¿Cuáles son las combinaciones más frecuentes de género y rating en el dataset?
    (Sugerencia: utilizar `value_counts` con `subset=["genre", "rating"]` después de aplicar `explode()`).



### Bonus: Análisis de duplicados y limpieza

12. ¿Existen películas con el mismo nombre (`title`) pero con distinto año de lanzamiento (`release_year`)?
13. ¿Cuántos títulos únicos hay en total en la columna `title`?





In [ ]:
#FIXME